In [2]:
import numpy as np
import tensorflow as tf
import os
import copy
from math import pi
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
from pandas import DataFrame as DF
from models import AP_ResNet, ResNet, VGG_branch

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from classifier import preprocessing as prep
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

2022-08-03 05:11:17.625394: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
class_names = ['asphalt','bicycle','block','floor','ground']
bound = 414

In [3]:
def heatmap(matrix,title):
    df=DF(matrix, index = class_names, columns = class_names)
    plt.figure(figsize=(10,10))
    sns.heatmap(df, annot=True)
    plt.tick_params(axis='x', top=True, labeltop = True,bottom=False, labelbottom=False)
    plt.xticks(np.arange(0.5, len(df.columns), 1), df.columns)
    plt.yticks(np.arange(0.5, len(df.index), 1), df.index)
    plt.xlabel("Prediction",position = (0.5,1.0+0.05))
    plt.ylabel("Object")
    plt.title(title)

In [4]:
label2idx_Dict = {
                'asphalt' : 0,
                'bicycle' : 1,
                'block' : 2,
                'floor' : 3,
                'ground' : 4,
            }

idx2label_Dict = {
    0 : 'asphalt',
    1 : 'bicycle',
    2 : 'block',
    3 : 'floor',
    4 : 'ground',
}

In [5]:
dir_path = './road_data'
def readNpy(dir_path):
    class_num = len(idx2label_Dict)

    Asphalt = list()
    Bicycle = list()
    Block = list()
    Floor = list()
    Ground = list()
    Asphalt = np.array(Asphalt)
    Bicycle = np.array(Bicycle)
    Block = np.array(Block)
    Floor = np.array(Floor)
    Ground = np.array(Ground)
    

    for dir in os.listdir(dir_path):
        d_path = os.path.join(dir_path, dir)
        file_list = os.listdir(d_path)
        for file in file_list:
            file_path = os.path.join(d_path, file)
            if dir == idx2label_Dict[0] :
                if len(Asphalt) == 0:
                    Asphalt = np.load(file_path, allow_pickle=True)
                else :
                    Asphalt = np.append(Asphalt, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[1]:
                if len(Bicycle) == 0:
                    Bicycle = np.load(file_path, allow_pickle=True)
                else :
                    Bicycle = np.append(Bicycle, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[2]:
                if len(Block) == 0:
                    Block = np.load(file_path, allow_pickle=True)
                else:
                    Block = np.append(Block, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[3]:
                if len(Floor) == 0:
                    Floor = np.load(file_path, allow_pickle=True)
                else:
                    Floor = np.append(Floor, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[4]:
                if len(Ground) == 0:
                    Ground = np.load(file_path, allow_pickle=True)
                else:
                    Ground = np.append(Ground, np.load(file_path), axis = 0)

    bound = Asphalt.shape[1]

    Ground_label = np.full((Ground.shape[0], class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['ground']])
    Asphalt_label = np.full((Asphalt.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['asphalt']])
    Bicycle_label = np.full((Bicycle.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['bicycle']])
    Block_label = np.full((Block.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['block']])
    Floor_label = np.full((Floor.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['floor']])

    Ground = np.concatenate((Ground, Ground_label), axis=1)
    Asphalt = np.concatenate((Asphalt, Asphalt_label), axis=1)
    Bicycle = np.concatenate((Bicycle, Bicycle_label), axis=1)
    Block = np.concatenate((Block, Block_label), axis=1)
    Floor = np.concatenate((Floor, Floor_label), axis=1)
    
    array = Asphalt
    array = np.append(array, Bicycle, axis = 0)
    array = np.append(array, Block, axis = 0)
    array = np.append(array, Floor, axis = 0)
    array = np.append(array, Ground, axis = 0)
    s = np.arange(array.shape[0])
    np.random.shuffle(s)
    array_s = array[s]

    X = array_s[:,:bound]
    Y = np.real(array_s[:,bound:])
    return copy.deepcopy(X), copy.deepcopy(Y)

X, Y = readNpy(dir_path)

In [6]:
def intervalMean(arr, num):
    meaned_arr = []
    meaned_arr = np.array(meaned_arr)
    meaned_arr = np.mean(arr[0:num], axis = 0)
    meaned_arr = np.expand_dims(meaned_arr, axis = 0)
    for i in range(num, len(arr), num):
        # print(arr.shape)
        if (i+num) > len(arr):
            temp = np.mean(arr[i : (len(arr) + 1)], axis = 0)
            temp = np.expand_dims(temp, axis = 0)
            meaned_arr = np.concatenate((meaned_arr, temp), axis = 0)
        else :
            # print(np.mean(arr[i:i+num], axis = 0).shape)
            temp = np.mean(arr[i : (i + num)], axis = 0)
            temp = np.expand_dims(temp, axis = 0)
            # print(temp.shape)
            meaned_arr = np.concatenate((meaned_arr, temp), axis = 0)
    return meaned_arr
    
def readNpyMean(dir_path, num):
    class_num = len(idx2label_Dict)

    Asphalt = list()
    Bicycle = list()
    Block = list()
    Floor = list()
    Ground = list()
    Asphalt = np.array(Asphalt)
    Bicycle = np.array(Bicycle)
    Block = np.array(Block)
    Floor = np.array(Floor)
    Ground = np.array(Ground)
    

    for dir in os.listdir(dir_path):
        d_path = os.path.join(dir_path, dir)
        file_list = os.listdir(d_path)
        for file in file_list:
            file_path = os.path.join(d_path, file)
            if dir == idx2label_Dict[0] :
                if len(Asphalt) == 0:
                    Asphalt = np.load(file_path, allow_pickle=True)
                else :
                    Asphalt = np.append(Asphalt, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[1]:
                if len(Bicycle) == 0:
                    Bicycle = np.load(file_path, allow_pickle=True)
                else :
                    Bicycle = np.append(Bicycle, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[2]:
                if len(Block) == 0:
                    Block = np.load(file_path, allow_pickle=True)
                else:
                    Block = np.append(Block, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[3]:
                if len(Floor) == 0:
                    Floor = np.load(file_path, allow_pickle=True)
                else:
                    Floor = np.append(Floor, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[4]:
                if len(Ground) == 0:
                    Ground = np.load(file_path, allow_pickle=True)
                else:
                    Ground = np.append(Ground, np.load(file_path), axis = 0)

    bound = Asphalt.shape[1]
    Asphalt_mean = intervalMean(Asphalt, num)
    Bicycle_mean = intervalMean(Bicycle, num)
    Block_mean = intervalMean(Block, num)
    Floor_mean = intervalMean(Floor, num)
    Ground_mean = intervalMean(Ground, num)

    Ground_label = np.full((Ground_mean.shape[0], class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['ground']])
    Asphalt_label = np.full((Asphalt_mean.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['asphalt']])
    Bicycle_label = np.full((Bicycle_mean.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['bicycle']])
    Block_label = np.full((Block_mean.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['block']])
    Floor_label = np.full((Floor_mean.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['floor']])

    Ground = np.concatenate((Ground_mean, Ground_label), axis=1)
    Asphalt = np.concatenate((Asphalt_mean, Asphalt_label), axis=1)
    Bicycle = np.concatenate((Bicycle_mean, Bicycle_label), axis=1)
    Block = np.concatenate((Block_mean, Block_label), axis=1)
    Floor = np.concatenate((Floor_mean, Floor_label), axis=1)
    
    array = Asphalt
    array = np.append(array, Bicycle, axis = 0)
    array = np.append(array, Block, axis = 0)
    array = np.append(array, Floor, axis = 0)
    array = np.append(array, Ground, axis = 0)
    s = np.arange(array.shape[0])
    np.random.shuffle(s)
    array_s = array[s]

    X = array_s[:,:bound]
    Y = np.real(array_s[:,bound:])
    return copy.deepcopy(X), copy.deepcopy(Y)

In [7]:
maximum = np.max(np.abs(X))

In [8]:
def seperater(arr):
    global maximum
    pre_data = arr
    amp = np.abs(pre_data)
    amp = amp / maximum
    phs = np.angle(pre_data)
    # phs = (phs - (- pi)) / (pi - (- pi))
    sin = np.sin(phs)
    sin = (sin + 1) / 2
    seperated_data = np.stack((amp.T,sin.T), axis=0)
    seperated_data = np.expand_dims(seperated_data, axis=0)
    return np.array(seperated_data)

In [9]:
def dataSeperator(arr):
    temp = copy.deepcopy(seperater(arr[0]))
    for i in range(1, len(arr)):
        temp = np.concatenate((temp, seperater(arr[i])), axis=0)
    return temp

In [10]:
Split_X = dataSeperator(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(Split_X, Y, test_size = 0.3)

In [13]:
checkpoint_filepath = 'ap_model'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = './model/' + checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only = True,
    save_weigths_only = False,
)
log_dir = './logs/fit/'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [13]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, predictions)
  # print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss)))
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
  predictions = model(images, training=False)

  t_loss = loss_object(labels, predictions)
  test_loss(t_loss)
  test_accuracy(labels, predictions)
  
ap_model = AP_ResNet()
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name = 'test_accuracy')

2022-08-02 06:20:31.217102: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-02 06:20:31.732483: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22304 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d8:00.0, compute capability: 8.6


In [14]:
EPOCHS = 10

In [15]:
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
for epoch in range(EPOCHS):
  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    train_step(ap_model, x_batch_train, y_batch_train, loss_fn, optimizer, train_loss, train_accuracy)

  for step ,(x_batch_test, y_batch_test) in enumerate(val_dataset):
    test_step(ap_model, x_batch_test, y_batch_test, loss_fn, test_loss, test_accuracy)

  template = "Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}, Test Loss: {:.4f}, Test Accuracy: {:.4f}"
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

In [ ]:
def train(model, X, Y, history_dict):
    Epoch = 100
    callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = './model/' + 'LSTM',
        monitor='val_accuracy',
        mode='max',
        save_best_only = True,
        save_weigths_only = False,
    )
    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.CategoricalCrossentropy()
    model.compile(optimizer = optimizer , loss = loss, metrics = ['accuracy', 'loss'])
    history = model.fit(X, Y, batch_size = 64, )


In [15]:
accuracy_dict = dict()
num = 15
for i in range(1,num+1):
    @tf.function
    def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)

        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss(loss)
        train_accuracy(labels, predictions)
    # print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss)))
    @tf.function
    def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
        predictions = model(images, training=False)

        t_loss = loss_object(labels, predictions)
        test_loss(t_loss)
        test_accuracy(labels, predictions)
    print('average of %d each' % (i))
    X, Y =  readNpyMean(dir_path, i)
    maximum = np.max(np.abs(X))
    # maximum = 1
    Split_X = dataSeperator(X)
    X_train, X_test, y_train, y_test = train_test_split(Split_X, Y, test_size = 0.3)
    ap_model = AP_ResNet()
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam()

    train_loss = tf.keras.metrics.Mean(name = 'train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name = 'train_accuracy')

    test_loss = tf.keras.metrics.Mean(name = 'test_loss')
    test_accuracy = tf.keras.metrics.CategoricalAccuracy(name = 'test_accuracy')
    EPOCHS = 200
    batch_size = 32
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))
    val_dataset = val_dataset.batch(batch_size)

    train_accuracy_arr = list()
    test_accuracy_arr = list()
    for epoch in range(EPOCHS):
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            train_step(ap_model, x_batch_train, y_batch_train, loss_fn, optimizer, train_loss, train_accuracy)

        for step ,(x_batch_test, y_batch_test) in enumerate(val_dataset):
            test_step(ap_model, x_batch_test, y_batch_test, loss_fn, test_loss, test_accuracy)
        train_accuracy_arr.append(train_accuracy.result())
        test_accuracy_arr.append(test_accuracy.result())
        template = "Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}, Test Loss: {:.4f}, Test Accuracy: {:.4f}"
        print(template.format(epoch+1,
                                train_loss.result(),
                                train_accuracy.result() * 100,
                                test_loss.result(),
                                test_accuracy.result() * 100))
    accuracy_dict[i] = {
        'train_acc' : np.max(train_accuracy),
        'test_acc' : np.max(test_accuracy),
    }

average of 1 each
Epoch 1, Loss: 7.4142, Accuracy: 23.9881, Test Loss: 1.6055, Test Accuracy: 25.4167
Epoch 2, Loss: 4.5213, Accuracy: 23.8988, Test Loss: 1.6019, Test Accuracy: 25.4167
Epoch 3, Loss: 3.5471, Accuracy: 24.7024, Test Loss: 1.5987, Test Accuracy: 24.0741
Epoch 4, Loss: 3.0566, Accuracy: 25.1935, Test Loss: 1.5959, Test Accuracy: 23.4028
Epoch 5, Loss: 2.7628, Accuracy: 25.5833, Test Loss: 1.5935, Test Accuracy: 23.0000
Epoch 6, Loss: 2.5696, Accuracy: 25.9722, Test Loss: 1.5915, Test Accuracy: 22.7315
Epoch 7, Loss: 2.4270, Accuracy: 26.1905, Test Loss: 1.5900, Test Accuracy: 22.5397
Epoch 8, Loss: 2.3216, Accuracy: 26.2574, Test Loss: 1.5885, Test Accuracy: 22.3958
Epoch 9, Loss: 2.2376, Accuracy: 26.3029, Test Loss: 1.5872, Test Accuracy: 22.2840
Epoch 10, Loss: 2.1696, Accuracy: 26.3750, Test Loss: 1.5860, Test Accuracy: 22.1944
Epoch 11, Loss: 2.1146, Accuracy: 26.4610, Test Loss: 1.5851, Test Accuracy: 22.1212
Epoch 12, Loss: 2.0686, Accuracy: 26.4732, Test Loss: 1.

In [19]:
print(np.min(np.abs(X)))

0.0


In [24]:
accuracy_dict = dict()
for i in range(1,16):
    @tf.function
    def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)

        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss(loss)
        train_accuracy(labels, predictions)
    # print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss)))
    @tf.function
    def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
        predictions = model(images, training=False)

        t_loss = loss_object(labels, predictions)
        test_loss(t_loss)
        test_accuracy(labels, predictions)
    print('average of %d each' % (i))
    X, Y =  readNpyMean(dir_path, i)
    maximum = np.max(np.abs(X))
    # maximum = 1
    Split_X = dataSeperator(X)
    X_train, X_test, y_train, y_test = train_test_split(Split_X, Y, test_size = 0.3)
    ap_model = ResNet()
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam()

    train_loss = tf.keras.metrics.Mean(name = 'train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name = 'train_accuracy')

    test_loss = tf.keras.metrics.Mean(name = 'test_loss')
    test_accuracy = tf.keras.metrics.CategoricalAccuracy(name = 'test_accuracy')
    EPOCHS = 50
    batch_size = 32
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))
    val_dataset = val_dataset.batch(batch_size)

    train_accuracy_arr = list()
    test_accuracy_arr = list()
    for epoch in range(EPOCHS):
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            train_step(ap_model, x_batch_train, y_batch_train, loss_fn, optimizer, train_loss, train_accuracy)

        for step ,(x_batch_test, y_batch_test) in enumerate(val_dataset):
            test_step(ap_model, x_batch_test, y_batch_test, loss_fn, test_loss, test_accuracy)
        train_accuracy_arr.append(train_accuracy.result())
        test_accuracy_arr.append(test_accuracy.result())
        template = "Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}, Test Loss: {:.4f}, Test Accuracy: {:.4f}"
        print(template.format(epoch+1,
                                train_loss.result(),
                                train_accuracy.result() * 100,
                                test_loss.result(),
                                test_accuracy.result() * 100))
    accuracy_dict[i] = {
        'train_acc' : np.max(train_accuracy),
        'test_acc' : np.max(test_accuracy),
    }

average of 1 each
Epoch 1, Loss: 1.8366, Accuracy: 40.5952, Test Loss: 1.6303, Test Accuracy: 23.8889
Epoch 2, Loss: 1.5611, Accuracy: 44.1964, Test Loss: 1.6048, Test Accuracy: 23.8889
Epoch 3, Loss: 1.4485, Accuracy: 46.2698, Test Loss: 1.5821, Test Accuracy: 24.2130
Epoch 4, Loss: 1.3968, Accuracy: 47.0387, Test Loss: 1.5610, Test Accuracy: 27.3264
Epoch 5, Loss: 1.3573, Accuracy: 47.7381, Test Loss: 1.5389, Test Accuracy: 29.1111
Epoch 6, Loss: 1.3317, Accuracy: 48.0754, Test Loss: 1.5238, Test Accuracy: 29.9074
Epoch 7, Loss: 1.3091, Accuracy: 48.5884, Test Loss: 1.5008, Test Accuracy: 31.3889
Epoch 8, Loss: 1.2919, Accuracy: 48.9211, Test Loss: 1.4903, Test Accuracy: 31.6319
Epoch 9, Loss: 1.2785, Accuracy: 49.3915, Test Loss: 1.4832, Test Accuracy: 32.1296
Epoch 10, Loss: 1.2631, Accuracy: 49.7024, Test Loss: 1.4700, Test Accuracy: 33.2500
Epoch 11, Loss: 1.2515, Accuracy: 50.0920, Test Loss: 1.4582, Test Accuracy: 34.1162
Epoch 12, Loss: 1.2425, Accuracy: 50.3720, Test Loss: 1.

In [11]:
model = VGG_branch(Split_X.shape[2])

2022-08-02 06:59:00.186680: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-02 06:59:00.678162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22304 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d8:00.0, compute capability: 8.6


(None, 414, 1)
414
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 414)]     0           []                               
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 414)         0           ['input_1[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.__operators__.getitem_1 (Sl  (None, 414)         0           ['input_1[0][0]']                
 icingOpLambda)                                                                                   
                                                                           

In [14]:
history=model.fit(
    Split_X,
    Y,
    batch_size = 32,
    epochs=100,
    validation_split = 0.3,
    shuffle = True,
    callbacks = [callback, tensorboard_callback]
    )

Epoch 1/100


2022-08-02 06:59:21.040976: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8301
2022-08-02 06:59:22.469548: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


53/53 [==============================] - ETA: 0s - loss: 1.5777 - accuracy: 0.2381

INFO:tensorflow:Assets written to: ./model/ap_model/assets


INFO:tensorflow:Assets written to: ./model/ap_model/assets


53/53 [==============================] - 29s 484ms/step - loss: 1.5777 - accuracy: 0.2381 - val_loss: 1.5660 - val_accuracy: 0.2389
Epoch 2/100
51/53 [===========================>..] - ETA: 0s - loss: 1.5288 - accuracy: 0.3045

INFO:tensorflow:Assets written to: ./model/ap_model/assets


INFO:tensorflow:Assets written to: ./model/ap_model/assets


53/53 [==============================] - 13s 255ms/step - loss: 1.5304 - accuracy: 0.3030 - val_loss: 1.5590 - val_accuracy: 0.2611
Epoch 3/100
53/53 [==============================] - 17s 324ms/step - loss: 1.5431 - accuracy: 0.3042 - val_loss: 1.5657 - val_accuracy: 0.2375
Epoch 4/100
53/53 [==============================] - 25s 471ms/step - loss: 1.5780 - accuracy: 0.2440 - val_loss: 1.5649 - val_accuracy: 0.2444
Epoch 5/100
53/53 [==============================] - 7s 135ms/step - loss: 1.5718 - accuracy: 0.2464 - val_loss: 1.5678 - val_accuracy: 0.2444
Epoch 6/100
53/53 [==============================] - 23s 449ms/step - loss: 1.5734 - accuracy: 0.2589 - val_loss: 1.5649 - val_accuracy: 0.2611
Epoch 7/100
53/53 [==============================] - 7s 131ms/step - loss: 1.5709 - accuracy: 0.2458 - val_loss: 1.5656 - val_accuracy: 0.2375
Epoch 8/100
53/53 [==============================] - 7s 129ms/step - loss: 1.5667 - accuracy: 0.2643 - val_loss: 1.5671 - val_accuracy: 0.2375
Epoch 9